file = open("data/cornell movie-dialogs corpus/movie_lines.txt", "r",encoding='utf-8', errors='ignore')
data = file.read()

from nltk.corpus import wordnet
from nltk.metrics import edit_distance
import re
import enchant
from nltk.tokenize import word_tokenize

replacement_patterns = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'cannot'),
    (r'i\'m', 'i am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are'),
    (r'(\w+)\'d', '\g<1> would')
]

class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
        
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        return s
    

class SpellingReplacer(object):
    def __init__(self, dict_name="en", max_dist=2, min_word_length=1):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = max_dist
        self.min_word_length = min_word_length
        
    def replace(self, word):
        if self.spell_dict.check(word):
            return word
        if len(word) < self.min_word_length:
            return word
        suggestions = self.spell_dict.suggest(word)
        if suggestions and edit_distance(word, suggestions[0]) <= self.max_dist:
            return suggestions[0]
        else:
            return word

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()
        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())
        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None
        
    def replace_negations(self, sent):
        i, length = 0, len(sent)
        words = []
        while i < length:
            word = sent[i]
            if word == "not" and i+1 < length:
                ant = self.replace(sent[i+1])
                if ant:
                    words.append(ant)
                    i += 2
                    continue
            words.append(word)
            i += 1
        return words


class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
        
    def replace(self, word):
        if wordnet.synsets(word):
            return word
        repl_word = self.repeat_regexp.sub(self.repl, word)
        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word
        
def clean_text(text, replace_neg=False, use_unknown=False, dict_name="en", max_dist=2, min_word_length=1):
    rep = RepeatReplacer()
    ant = AntonymReplacer()
    reg = RegexpReplacer()
    spell = SpellingReplacer(dict_name, max_dist, min_word_length)
    # Get rid of some abbreviations
    text = reg.replace(text)
    words = word_tokenize(text)
    if replace_neg:
        words = ant.replace_negations(words)
    words = [spell.replace(word) for word in words]
    if not use_unknown:
        spell_dict = enchant.Dict(dict_name)
        words = [word for word in words if spell_dict.check(word)]
    return words

import re
from tqdm import tqdm

def clean_movie_data(data):
    pattern = " \+\+\+\$\+\+\+ u[0-9] \+\+\+\$\+\+\+ m[0-9] \+\+\+\$\+\+\+ \w* \+\+\+\$\+\+\+ "
    lines = re.split(pattern, data.lower())
    line_words = []
    for line in tqdm(lines):
        try:
            cleaned_line = clean_text(line, max_dist=0)
            if len(cleaned_line) >= 1:
                line_words.append(cleaned_line)
        except KeyboardInterrupt:
            break
    return line_words

In [1]:
%%time
import pandas as pd
# texts = clean_movie_data(data)
df = pd.read_csv("data/All-seasons.csv")

CPU times: user 332 ms, sys: 336 ms, total: 668 ms
Wall time: 297 ms


In [2]:
df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


In [3]:
texts = df["Line"].values

In [4]:
texts[:10]

array(['You guys, you guys! Chef is going away. \n',
       'Going away? For how long?\n', 'Forever.\n', "I'm sorry boys.\n",
       "Chef said he's been bored, so he joining a group called the Super Adventure Club. \n",
       'Wow!\n',
       'Chef?? What kind of questions do you think adventuring around the world is gonna answer?!\n',
       "What's the meaning of life? Why are we here?\n",
       "I hope you're making the right choice.\n",
       "I'm gonna miss him.  I'm gonna miss Chef and I...and I don't know how to tell him! \n"], dtype=object)

In [22]:
from sklearn import preprocessing
import nltk
import numpy as np

from hedgeable_ai.functions.preprocessing.basic_processor import BasicProcessor
from hedgeable_ai.functions.preprocessing.clean_text import clean_text


class Word2IndexProcessor(BasicProcessor):
    def __init__(self, texts, is_processed=False, num_additional=2,
                 max_dist=0, min_word_length=1,replace_neg=False, use_unknown=False):
        _texts = []
        lengths = []
        self.num_additional = num_additional
        for text in texts:
            if is_processed:
                words = text
            else:
                words = clean_text(text.lower(), max_dist=max_dist,
                                   min_word_length=min_word_length,
                                   replace_neg=replace_neg, use_unknown=use_unknown)
            if len(words) > 0:
                _texts.extend(words)
                lengths.append(len(words))
        lengths = list(np.cumsum(lengths))
        lengths.insert(0, 0)
        self.encoder = preprocessing.LabelEncoder()
        # 0 adn 1 are taken for padding and <eos> respectively
        indices = self.encoder.fit_transform(_texts) + self.num_additional
        # split to sentences
        self.data = np.array([indices[lengths[i]:lengths[i+1]] for i in range(len(lengths) - 1)])
        
    def encode(self, text):
        words = clean_text(text.lower(), max_dist=0, min_word_length=1,replace_neg=False, use_unknown=False)
        words = [word for word in words if word in self.encoder.classes_]
        if words:
            index = np.array(self.encoder.transform(words)) + self.num_additional
        else:
            index = np.array([])
        return index
    
    def decode(self, index):
        return [self.encoder.inverse_transform(i-self.num_additional) for i in index if i >=self.num_additional]    
            
    
    def batch_process_test(self, X, y=None):
        if y is None:
            return np.array([self.encode(x_i) for x_i in X])
        else:
            return np.array([self.encode(x_i) for x_i in X]), np.array(y)
        
    @property
    def vocab_size(self):
        return len(self.encoder.classes_)

    def batch_padding(self, inputs, max_sequence_length=None, pad_idx=0):
        """
        Args:
            inputs:
                list of sentences (integer lists)
            max_sequence_length:
                integer specifying how large should `max_time` dimension be.
                If None, maximum sequence length would be used
    
        Returns:
            inputs_time_major:
                input sentences transformed into time-major matrix 
                (shape [max_time, batch_size]) padded with 0s
            sequence_lengths:
                batch-sized list of integers specifying amount of active 
                time steps in each input sequence
        """
    
        sequence_lengths = [len(seq) for seq in inputs]
        batch_size = len(inputs)
    
        if max_sequence_length is None:
            max_sequence_length = max(sequence_lengths)
        sequence_lengths = [min(length, max_sequence_length) for length in sequence_lengths]
        max_sequence_length = max(sequence_lengths)
        inputs_batch_major = pad_idx * np.ones([batch_size, max_sequence_length], dtype=np.int32)
    
        for i, seq in enumerate(inputs):
            seq = seq[:max_sequence_length]
            for j, element in enumerate(seq):
                inputs_batch_major[i, j] = element

        return inputs_batch_major, sequence_lengths

In [39]:
%%time
# from hedgeable_ai.functions.preprocessing.word2index import Word2IndexProcessor

processor = Word2IndexProcessor(texts[:], is_processed=False)

CPU times: user 49min 41s, sys: 39.5 s, total: 50min 21s
Wall time: 49min 27s


In [96]:
from hedgeable_ai.models.nn.ff.core import FeedForward

class BaseAttention(object):
    def __init__(self, conf=None):
        if conf is None:
            conf = {}
        for key in self.default_conf.keys():
            if key not in conf.keys():
                conf[key] = self.default_conf[key]
        self.conf = conf
        
    def __call__(self, query, source_outputs, source_inputs, training=True):
        attentions, alignments = self._get_attentions(query, source_outputs, source_inputs, training)
        return attentions, alignments
    
    def _get_attentions(self, query, source_outputs, source_inputs, trainig):
        raise NotImplementedError()
    
class LuongAttention(BaseAttention):
    default_conf = {"local_scale":5, "attention_size":300}
    def __init__(self, conf=None):
        super().__init__(conf)
        self.local_scale = self.conf["local_scale"]
        self.attention_size = self.conf["attention_size"]
        self.middle_hidden = int(0.5 * self.attention_size)
        model_params = [{"name":"dense", "num_hidden":self.middle_hidden, "activation":tf.tanh}]
        self.local_ff = FeedForward(model_params, scope_name="local_ff")
        model_params = [{"name":"dense", "num_hidden":self.attention_size, "activation":tf.tanh}]
        self.attention_ff = FeedForward(model_params, scope_name="attention_ff")   
    
    def _get_global_attentions(self, query, source_outputs, source_inputs, training):
        """Calulate global alignments
        
        Args:
            query: tensor with shape=(batch, output_size), decoder RNN output
            source_outputs: tensor with shape=(batch, time_length, output_size), 
                entire outputs of encoder RNN
            source_inputs: tensor with shape=(batch, time_length)
            training: tensor bool
            
        Returns:
            alignments with shape = (batch, time_length)
        """
        query = tf.expand_dims(query, 1)
        scale = get_variable("global_scale", [],
                                     dtype=tf.float32,
                                     initializer=tf.ones_initializer)
        scale = scale / tf.constant(np.sqrt(self.attention_size))
        _score = tf.matmul(query, source_outputs, transpose_b=True)
        _score = tf.squeeze(_score, [1]) #shape=(batch, output_size)
        _alignments = tf.nn.softmax(_score)
        # We will not put attention to padded input
        weights = tf.cast(source_inputs >= 1, tf.float32)
        _alignments = weights * _alignments
        length = tf.shape(_alignments)[1]
        alignments = _alignments / tf.tile(tf.reduce_sum(_alignments, 1, keep_dims=True), [1, length])
        return alignments
    
    def _get_local_attentions(self, query, source_outputs, source_inputs, training):
        """Calulate local alignments
        
        Args:
            query: tensor with shape=(batch, output_size), decoder RNN output
            source_outputs: tensor with shape=(batch, time_length, output_size), 
                entire outputs of encoder RNN
            source_inputs: tensor with shape=(batch, time_length)
            training: tensor bool
            
        Returns:
            alignments with shape = (batch, time_length)
        """
        batch_size, length, output_size = tf.unstack(tf.shape(source_outputs))
        scale = get_variable("local_scale", [],
                             dtype=tf.float32,
                             initializer=tf.ones_initializer)
        scale = scale / tf.constant(np.sqrt(self.middle_hidden))
        u = self.local_ff(query, training) / scale
        # Consider only non zero index range, the size will be (batch, 1)
        S = tf.cast(tf.reduce_sum(tf.cast(source_inputs >= 1, tf.float32), axis=1, keep_dims=True), tf.float32)
        v_local = tf.get_variable("v_local", [self.middle_hidden], dtype=tf.float32)
        v_local = tf.expand_dims(v_local, [1])
        # Size will be [batch_size, 1]
        pt = S * tf.nn.sigmoid(tf.matmul(u, v_local))
        pt = tf.tile(pt, [1, length])
        # pt has shap=(batch, length) and values are the same along time axis
        sig = tf.constant(0.5) * self.local_scale
        # Index value will be in [1, S]
        index = tf.cast(tf.range(tf.constant(1), length + tf.constant(1)), tf.float32)
        index = tf.expand_dims(index, [0])
        index = tf.tile(index, [batch_size, 1])
        # Size of pt and  index has to be [batch_size, length]
        alignments = tf.exp(-0.5 * (pt - index)**2 / sig**2)
        return alignments
    
    def _get_attentions(self, query, source_outputs, source_inputs, training):
        local_alignments = self._get_local_attentions(query, source_outputs, source_inputs, training)
        global_alignments = self._get_global_attentions(query, source_outputs, source_inputs, training)
        alignments = global_alignments * local_alignments
        _alignments = tf.expand_dims(alignments, 1)
        context = tf.matmul(_alignments, source_outputs)
        # context shape is (batch_size, 1, output_size)
        context = tf.squeeze(context, [1])
        pre_attentions = tf.concat((context, query), axis=1)
        attentions = self.attention_ff(pre_attentions, training)
        return attentions, alignments

In [97]:
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.seq2seq as seq2seq
from hedgeable_ai.models.nn import BaseModel, get_shape, get_length
import tensorflow.contrib.rnn as rnn

import tensorflow as tf

from hedgeable_ai.models.nn.params import nn_is_logit
from hedgeable_ai.models.nn import BaseNN, get_shape

from hedgeable_ai.models.nn.rnn import get_cell

count = 0
class DialogueAgent(BaseNN):
    def __init__(self, processor, emb_size=300, maxlen=20,
                 conf=None, additional_length=3, clip_val=5.0, 
                 is_attention=False, *args, **kwargs):
        self.emb_size = emb_size
        self.clip_val = clip_val
        # leave index 0 for padding and 1 for  <eos>
        self.vocab_size = processor.vocab_size + 2
        self.maxlen = maxlen
        self.additional_length = additional_length
        self.is_attention = is_attention
        super().__init__(processor=processor, conf=conf, *args, **kwargs)
        
    def _build_graph(self):
        """Build tensorflow graph
        
        Note:
            You build graphs for output and input, which will be used 
            for training and prediction.
        """
        # Build Basic Netwoiork
        self.attention_model = LuongAttention(self.conf)
        W_output = tf.Variable(
            tf.random_uniform([self.attention_model.attention_size, self.vocab_size], -0.1, 0.1), dtype=tf.float32)
        b_output = tf.Variable(tf.zeros([self.vocab_size]), dtype=tf.float32)
        
        self.encoder_input = tf.placeholder(tf.int32, shape=(None, None), name="encoder_input")
        self.decoder_target = tf.placeholder(tf.int32, shape=(None, None), name="decoder_target")
        self.training = tf.placeholder(tf.bool, (), name="trainig")
        encoder_length = get_length(self.encoder_input)
        decoder_length = get_length(self.decoder_target)
        # Encoder
        encoder_cell = get_cell(self.conf["model"])
        embeddings = tf.Variable(
            tf.random_uniform([self.vocab_size, self.emb_size], -0.1, 0.1, dtype=tf.float32))
        encoder_input_embedded = tf.nn.embedding_lookup(embeddings, self.encoder_input)
        encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(
            encoder_cell, encoder_input_embedded, sequence_length=encoder_length, 
            dtype=tf.float32, time_major=False, scope="encoder")

        # Decoder
        decoder_cell = get_cell(self.conf["model"])
        batch_size, encoder_max_time = tf.unstack(tf.shape(self.encoder_input))
        decoder_output_size = self._get_output_size(self.conf["model"])
        # We use a decoder length 10 words longer than encoder
        max_decoder_length = encoder_length + tf.constant(self.additional_length)
        self.max_decoder_length = max_decoder_length
        # Prepare for padding and EOS
        eos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='EOS')
        eos_step_embedded = tf.nn.embedding_lookup(embeddings, eos_time_slice)
        pad_time_slice = tf.ones([batch_size], dtype=tf.int32, name='PAD')
        pad_step_embedded = tf.zeros_like(eos_step_embedded)
        
        def loop_fn_initial():
            initial_elements_finished = (0 >= max_decoder_length)
            initial_input = eos_step_embedded
            initial_cell_state = encoder_final_state
            initial_cell_output  = None
            initial_loop_state = None
            return (initial_elements_finished,
                initial_input,
                initial_cell_state,
                initial_cell_output,
                initial_loop_state)
        
        def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):
            if self.is_attention:
                previous_output, alignment = self.attention_model(
                    previous_output, encoder_outputs, self.encoder_input, self.training)
            output_logits = tf.matmul(previous_output, W_output) + b_output
            prediction  = tf.argmax(output_logits, axis=1)
            def get_next_input():
                next_input = tf.nn.embedding_lookup(embeddings,  prediction)
                return next_input
            elements_finished = (time >= max_decoder_length)
            finished = tf.reduce_all(elements_finished)
            next_input = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
            state  = previous_state
            loop_state = previous_loop_state
            output = previous_output
            return (elements_finished, 
                next_input,
                state,
                output,
                loop_state)
        
        def loop_fn(time, previous_output, previous_state,  previous_loop_state):
            if previous_state is None:
                assert previous_output is None and previous_state is None
                return loop_fn_initial()
            else:
                return loop_fn_transition(time, previous_output, previous_state,  previous_loop_state)
        
        decoder_outputs_ta, decoder_final_state, _ = tf.nn.raw_rnn(decoder_cell, loop_fn)
        decoder_outputs = decoder_outputs_ta.stack()
        decoder_max_steps, decoder_batch_size, decoder_outputs_size = tf.unstack(tf.shape(decoder_outputs))
        self.max_steps = decoder_max_steps
        self.decoder_batch_size = decoder_batch_size
        decoder_outputs_size = self.attention_model.attention_size
        self.decoder_outputs = decoder_outputs
        decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_outputs_size))
        decoder_logits_flat = tf.matmul(decoder_outputs_flat, W_output) + b_output
        decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_steps, decoder_batch_size, self.vocab_size))
        decoder_logits = tf.transpose(decoder_logits, [1, 0, 2])
        self.decoder_prediction = tf.argmax(decoder_logits, 2)
        self.decoder_logits = decoder_logits
        
        # Optimization
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.one_hot(self.decoder_target, depth=self.vocab_size, dtype=tf.float32),
            logits=decoder_logits)
        # cross_entropy = (tf.nn.softmax(self.decoder_logits) - tf.one_hot(self.decoder_target, depth=self.vocab_size, dtype=tf.float32))**2
        # We do not optimize for negative labels
        # weights = tf.cast(self.decoder_target >= 0, tf.float32)
        # self.weights = weights
        # self.loss = tf.reduce_mean(cross_entropy * weights)
        self.loss = tf.reduce_mean(cross_entropy)
        
        self.learning_rate_op = self._get_learning_rate()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            self.optimizer = self._get_optimizer(self.optimizer_name, self.learning_rate_op, self.optimizer_conf)
            grads_vars = self.optimizer.compute_gradients(self.loss)
            clipped_grads_vars = [
                (tf.clip_by_norm(gv[0], clip_norm=self.clip_val), gv[1]) 
                for gv in grads_vars]
            self.train_step = self.optimizer.apply_gradients(clipped_grads_vars)
        
    def _optimize(self, batch_X, batch_y, *args, **kwargs):
        global count
        batch_X = [x[::-1] for x in batch_X[0]]
        batch_X, Xlen = self.processor.batch_padding(batch_X, self.maxlen)
        length = np.max(Xlen) + self.additional_length
        batch_y = self._batch_padding(batch_y, length)
        feed_dict = {self.encoder_input: batch_X,
                     self.decoder_target: batch_y,
                     self.training: False}
        
        # print(np.argmax(logits, 2))
        # print("batch_y", batch_y)
        # print(decoder_batch)
        # print(one_hot)
        if count % 1000 == 0:
            logits = self.sess.run(self.decoder_logits, feed_dict=feed_dict)
            # alignments = self.sess.run(self.alignments, feed_dict=feed_dict)
            decoder_batch = self.sess.run(self.decoder_batch_size, feed_dict=feed_dict)
            print("predition", np.argmax(logits, 2))
            print("max", np.max(np.argmax(logits, 2)))
        # print("alignments", logits)
        # print("logits", logits)
        # print(np.max(np.argmax(logits, 2)))
        count += 1
        _, loss = self.sess.run([self.train_step, self.loss], feed_dict=feed_dict)
        return loss
    
    def _get_output_size(self, conf):
        if isinstance(conf, list) or isinstance(conf, tuple):
            x = conf[-1]
        else:
            x = conf
        return x["num_units"]
    
    def generate_sentences(self, sentences):
        X = [self.processor.encode(sentence) for sentence in sentences]
        # X = sentences
        X = [x_[::-1] for x_ in X]
        X, Xlen = self.processor.batch_padding(X, self.maxlen)
        feed_dict = {self.encoder_input: X,
                     self.training: True}
        word_idx = self.sess.run(self.decoder_prediction, feed_dict=feed_dict)
        logits =  self.sess.run(self.decoder_logits, feed_dict=feed_dict)
        print(word_idx)
        return [self.processor.decode(i) for i in word_idx], logits
    
    def _batch_padding(self, batch, length):
        EOS = 1
        PAD = 0
        padded_batch = []
        for x in batch:
            x = list(x)
            if len(x) > length:
                x = x[:length]
            elif len(x) < length:
                x.append(EOS)
            while len(x) < length:
                x.append(PAD)
            padded_batch.append(x)
        return np.array(padded_batch)

In [ ]:
import tensorflow as tf
import numpy as np


conf = {
        "learning_rate": .5,
        "learning_rate_minimum": 1e-3,
        "learning_rate_decay": 0.99,
        "learning_rate_decay_step": 1,
        "batch_size": 128,
        "model_dir": "./logs",
        "load_file_path": None,
        "save_file_path": None,
        "log_freq": 1,
        "optimizer":"gd",
        "model":[{"name":"lstm", "num_units":512},
                {"name":"lstm", "num_units":512},],
        # "model":[{"name":"lstm", "num_units":500},
        #         {"name":"lstm", "num_units":500}],
        "attention_size": 512
}

tf.reset_default_graph()
agent = DialogueAgent(processor, maxlen=10, conf=conf, additional_length=3, is_attention=False)

train_X = processor.data[:-1]
train_y = processor.data[1:]
agent.fit(train_X, train_y, num_epochs=10000, batch_bar=False, log_freq=1, batch_log_freq=100)

/home/tomoaki/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




  0%|          | 0/10000 [00:00<?, ?it/s]

Model saved in file: params/model.ckpt






  0%|          | 1/10000 [01:01<170:37:50, 61.43s/it]

predition [[10738    36     1 ...,     0     0     0]
 [17914 17914     1 ...,     0     0     0]
 [10738    36     1 ...,     0     0     0]
 ..., 
 [    0     1     1 ...,     0     0     0]
 [10738    36     1 ...,     0     0     0]
 [10738 17914     1 ...,     0     0     0]]
max 17914






  0%|          | 2/10000 [02:03<170:50:01, 61.51s/it]

...

predition [[ 7949  8492    36 ...,     0     0     0]
 [17914 17914     1 ...,     0     0     0]
 [ 7949  7949 10552 ..., 16139    36    36]
 ..., 
 [17496    36 17914 ...,     0     0     0]
 [ 7949  8492    36 ...,     0     0     0]
 [ 7949    36    36 ...,     0     0     0]]
max 17914






  5%|▍         | 482/10000 [8:21:50<164:14:02, 62.12s/it]



  5%|▍         | 483/10000 [8:22:51<163:52:36, 61.99s/it]

predition [[ 7949  8492 10552 ...,     0     0     0]
 [17877    36 17914 ...,     0     0     0]
 [17496     1     0 ...,     0     0     0]
 ..., 
 [ 7949  8492 10552 ...,     0     0     0]
 [17496  8492  8492 ...,     0     0     0]
 [ 7949     1 10552 ...,     0     0     0]]
max 17914






  5%|▍         | 484/10000 [8:23:53<163:45:11, 61.95s/it]



  5%|▍         | 485/10000 [8:24:55<163:54:46, 62.02s/it]

predition [[17877     1     0 ...,     0     0     0]
 [ 7949  8492     0 ...,     0     0     0]
 [ 7949  8492  8492 ...,    36    36     0]
 ..., 
 [17496     1     0 ...,     0     0     0]
 [ 7949    36 17914 ...,     0     0     0]
 [ 7949  8492     0 ...,     0     0     0]]
max 17914


In [94]:
prediction, logits = agent.generate_sentences(texts[:1000])

[[ 7949     1     1 ...,     0     0     0]
 [ 7949  8492     1 ...,     0     0     0]
 [   36    36     1 ...,     0     0     0]
 ..., 
 [ 7949  8492  8492 ...,     0     0     0]
 [ 7949  8492 10552 ...,     1     0     0]
 [ 7949  7949  7949 ...,    36    36    36]]


In [95]:
print(prediction)

[['i'], ['i', 'is'], ['.', '.'], ['i', '.', '.'], ['what'], ['i'], ['i', 'is', 'is', 'is', 'is', 'the', 'the', '.', '.'], ['i', 'is', 'not', '.', '.', '.', '.', '.'], ['i', '.', 'you', 'you', '.', '.', '.', '.', '.', '.'], ['you', 'you'], ['i', 'is', 'is', '.', '.', '.', '.'], ['what'], ['i'], ['i', '.'], ['i', 'is'], ['i', 'is'], ['oh', 'is'], ['what', '.'], ['oh', 'is', '.'], ['.', '.', '.', '.'], ['i'], ['you', 'is', 'is'], ['hey'], ['i', 'is'], ['oh'], ['oh', '.'], ['i', 'i', 'i'], ['i', 'is', 'is'], ['i', 'i', '.'], ['i', '.', '.'], ['oh', 'is', 'you'], ['i', 'is'], ['i', 'is'], ['what'], ['oh'], ['i', 'is', 'not', 'you', '.', 'the', 'the', 'you', 'the', 'the', '.', '.', '.'], ['.'], ['i', 'is', 'is', 'is', '.', '.'], ['i', '.', '.'], ['oh', 'is', 'you', '.'], ['i', '.', '.'], ['what'], ['i', '.', '.'], ['oh', 'is', 'you', '.', '.', '.'], ['oh', 'is', '.', 'is', 'is', 'you', 'you', 'you', '.', '.'], ['i', 'i', 'you', 'you', 'you', '.'], ['oh', '.', 'you', '.', '.'], ['oh', '.'], [

In [75]:
processor.vocab_size

17960

In [89]:
texts[0]

'You guys, you guys! Chef is going away. \n'